In [19]:
from matplotlib import pyplot as plt
import numpy as np
import torch.nn as nn
import torch
import torch.optim as optim

# edgeitems : 大きなテンソルは見にくいのでテンソルを表示する際に、先頭2つと末尾2つのみに制限する
# linewidth : テンソルを1行に表示する際の最大幅を指定します。テンソルがこの幅を超える場合、改行されます。
torch.set_printoptions(edgeitems=2, linewidth=75)

In [20]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [21]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

print(shuffled_indices)
train_indices, val_indices

tensor([ 1, 10,  9,  4,  7,  8,  6,  2,  0,  3,  5])


(tensor([ 1, 10,  9,  4,  7,  8,  6,  2,  0]), tensor([3, 5]))

In [22]:
t_u_train = t_u[train_indices]
t_u_val = t_u[val_indices]

t_c_train = t_c[train_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [23]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(
    linear_model.parameters(),
    lr=1e-2
)

In [24]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):

        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [ ]:
training_loop(n_epochs=3000, optimizer=optimizer, model=linear_model, loss_fn=nn.MSELoss(), t_u_train=t_un_train, t_u_val=t_un_val, t_c_train=t_c_train, t_c_val=t_c_val)
linear_model.weight, linear_model.bias

Epoch 1, Training loss 59.9256, Validation loss 221.3723
Epoch 1000, Training loss 4.1754, Validation loss 9.3497
Epoch 2000, Training loss 3.2059, Validation loss 3.8807
Epoch 3000, Training loss 3.1601, Validation loss 3.0148


(Parameter containing:
 tensor([[5.0941]], requires_grad=True),
 Parameter containing:
 tensor([-16.0014], requires_grad=True))